# Load pretrained model

In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained(
    "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
)
model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/hoang/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


# Prepare Dataset

In [2]:
import sys

sys.path.append("..")

from src.datamodule import WebDatasetConverter, VLSP2020ForPretrainingDataModule


def get_dataloader(dataset, batch_size: int = 4):
    dtm = VLSP2020ForPretrainingDataModule(
        dts, batch_size=batch_size, return_transcript=True
    )
    dtm.setup()

    return dtm.train_dataloader()


dts = WebDatasetConverter("../data/vlsp2020.tar").get_dataset()
dataloader = get_dataloader(dts, batch_size=2)


/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


# Inference

In [3]:
from typing import Tuple
from src.model.modules import Wav2Vec2Processor
from src.model import Wav2Vec2PretrainingModule
import torch


def speech_to_text(waveforms: Tuple[torch.Tensor, ...]):

    batched_waveforms, wavelengths = Wav2Vec2Processor()(waveforms)
    attention_masks = Wav2Vec2PretrainingModule._compute_attention_mask(wavelengths)

    logits = model(batched_waveforms, attention_mask=attention_masks).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    return processor.batch_decode(predicted_ids)


In [4]:
import re


def levenshtein_distance(source: Tuple[str], target: Tuple[str]):
    """
    Compute the Levenshtein distance between two sequences.
    """

    n, m = len(source), len(target)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        source, target = target, source
        n, m = m, n

    current_row = range(n + 1)  # Keep current and previous row, not entire matrix
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = (
                previous_row[j] + 1,
                current_row[j - 1] + 1,
                previous_row[j - 1],
            )
            if source[j - 1] != target[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    distance = current_row[n]

    del current_row
    del previous_row
    
    return distance


def word_error_rate(prediction: str, transcript: str):
    pattern = r"\W+"

    prediction = re.split(pattern, prediction)
    transcript = re.split(pattern, transcript)

    return levenshtein_distance(prediction, transcript) / len(transcript)


def character_error_rate(prediction: str, transcript: str):
    return levenshtein_distance(prediction, transcript) / len(transcript)


In [7]:
cer, wer = 0, 0
n_items = 0

for batch_idx, batch in enumerate(dataloader):
    transcripts, waveforms = batch
    predicted_transcripts = speech_to_text(waveforms)

    for predicted_transcript, transcript in zip(predicted_transcripts, transcripts):
        cer += character_error_rate(predicted_transcript, transcript)
        wer += word_error_rate(predicted_transcript, transcript)

        n_items += 1

    if batch_idx > 100:
        break

    del transcript
    del waveforms
    del predicted_transcript
    del batch

cer /= n_items
wer /= n_items

print(f"Character error rate: {cer:.2%}")
print(f"Word error rate: {wer:.2%}")


Character error rate: 25.47%
Word error rate: 32.33%


In [12]:
import pandas as pd

m = []

for batch_idx, batch in enumerate(dataloader):
    if batch_idx > 5:
        break

    transcripts, waveforms = batch
    predicted_transcripts = speech_to_text(waveforms)

    for predicted_transcript, transcript in zip(predicted_transcripts, transcripts):
        m.append(
            {
                "transcript": transcript,
                "predicted_transcript": predicted_transcript,
            }
        )


pd.set_option("display.max_colwidth", None)
pd.DataFrame.from_dict(m)


,transcript,predicted_transcript
0,trước khi bắt đầu khởi hành hiển dự định đạp xe vào bốn giờ ba mươi phút sáng hàng ngày,trước khi bắt đầu khởi hành hiện dự định đạp xe và bốn giờ ba mươi phút sáng hằng ngày
1,<unk> ngon,ớnđyvn
2,xin được cảm ơn những chia sẻ đầy cảm xúc của các khách mời đã giúp cho chúng tôi những người thuộc nhiều thế hệ được lớn lên trong hòa bình thêm trân trọng những mất mát hi sinh của thế hệ đi trước để có được một việt nam hòa bình giúp cho chúng tôi,xin được cảm ơn những chia sẻ đầy cảm xúc của các khách mời đã giúp cho chúng tôi những người thuộc nhiều thế hệ được lớn lên trong hòa bình thêm trân trọng những mất mát hi sinh của thấy đi trước để có được một việt nam hòa bình giúp cho chúng tôi
3,nguyễn trang đem thi thể chúa trịnh nộp cho quân tây sơn,nguyễn trang đem thi thể chúa trịnh nộp cho quân tây sơn
4,theo đó chủ mưu trong vụ án là bị cáo nguyễn minh hùng bị tuyên mười bảy năm tù với vai trò chỉ,theo đó chủ mưu trong vụ án là bị cáo nguyễn minh hùng bị tuyên mười bảy năm tù với vai trò chỉ
5,bây giờ sẽ là phần thi cuối cùng và cũng là <unk>,bây giờ sẽ là phần thi cuối cùng và cũn là
6,đồng thời lại nghe theo kiến nghị của quân sư ông cho đúc trở lại tiền ngũ thù khôi phục lưu thông tiền tệ trong địa bàn đem đến tiện lợi cho sinh hoạt của nhân dân không còn phải dùng xếp vải làm phương tiện thanh toán,đồng thời lại nghe theo kýn nghị của quân sư ông cho đúc trở lại tiền ngủ thù khôi phục lưu thông tiền tệ trong địa bàn đem đến tiền lợi cho sinh hoạt của nhân dân không còn phải dùng xếp vãi làm phương tiện thanh toán
7,chính quyền của tổng thống trăm nhằm cắt giảm các chương trình trợ cấp an sinh xã hội,chính quyền của tổng thống trămp nhắm cắt giảm các chương trình trơa cấp an sinh xã hộicc
8,văn học việt nam là sự tích hợp từ hai dòng văn học dân gian và văn học viết của những người dùng tiếng việt,văn học việt nam là sự tích hợp từ hai dòng văn học dân gian và văn học viết của những người dùng tiếng việt
9,ui tình,ừừừ
